In [20]:
# loading packages
import pandas as pd
import numpy as np
import scipy.stats as st
import math

In [21]:
# create constant varaiables
DATA = 'AB_test_data.csv'

In [22]:
# reading the csv file into a data frame
df = pd.read_csv(DATA)
df.head()

,purchase_TF,Variant,date,id
0,False,A,2019-11-08,0x25b44a
1,False,B,2020-08-27,0x46271e
2,False,A,2020-06-11,0x80b8f1
3,False,B,2020-08-22,0x8d736d
4,False,A,2020-08-05,0x96c9c8


In [23]:
# checking basic infomation of the data frame
print(df.shape)
print(df.isnull().sum())
print(df.dtypes)
# conclusion: no missing values

(130000, 4)
purchase_TF    0
Variant        0
date           0
id             0
dtype: int64
purchase_TF      bool
Variant        object
date           object
id             object
dtype: object


# Question1 A/B testing

## 1. Define hypotheses:

- Hypothesis:
Implementing alternative B can improve the conversion rates over applying alternative A


In [24]:
# spliting the data frame by the values of variable Variant
df_A = df[df['Variant'] == 'A']
print(df_A.shape)

(125000, 4)


In [25]:
df_B = df[df['Variant'] == 'B']
print(df_B.shape)
size_B = df.shape[0]
print(size_B)

(5000, 4)
130000


In [26]:
# Calculating the conversion rate of each group
def cal_conversion_rate(my_df, target_variable):
    """
    This function returns the conversion rate of a group
    
    """
    p = np.mean(my_df[target_variable].tolist())
    print('The conversion rate is:'+ str(p))
    return p
                
con_A =  float(cal_conversion_rate(df_A,'purchase_TF'))
con_B = float(cal_conversion_rate(df_B, 'purchase_TF'))              
            

The conversion rate is:0.149616
The conversion rate is:0.1766


In [27]:
# Conducting the A/B testing 

def do_abtesting(pa, pb, b_size, confidence):
    """
    pa: 
    pb:
    b_size:
    confidence: 
    """
    # calculating the Z-score of variant B
    stv_b = math.sqrt(pa*(1-pa)/b_size)
    z_b = (pb-pa)/stv_b
    # comparing the z-score to the value of statistical significance at the level of a
    # one-tailed test
    z_confi = st.norm.ppf(confidence)
    if z_b > z_confi:
        print('z-score of variant b: %s > z-score of the confidence rate: %s' %(z_b, z_confi))
        print()
        print('We can reject null hypothesis given that confidence rate.' )
        print()
        print('Therefore, implementing alternative B can improve the conversion rates over applying alternative A.')
    else:
        print('z-score of variant b: %s < z-score of the confidence rate: %s' %(z_b, z_confi))
        print()
        print('We fail to reject null hypothesis given that confidence rate.')
        print()
        print('There is no evidence to support the claim that implementing alternative B can improve the conversion rates over applying alternative A.')
       

In [28]:
# supposing we conduct the testing at the confidence level of 0.95
do_abtesting(pa = con_A, pb = con_B, b_size = size_B, confidence = 0.95)

z-score of variant b: 27.276047893577964 > z-score of the confidence rate: 1.6448536269514722

We can reject null hypothesis given that confidence rate.

Therefore, implementing alternative B can improve the conversion rates over applying alternative A.


# Question 2 

## Q2-1 

### Calculate the optimal sample size for a 95% confidence rate and test with 80% power.

In [29]:
# First, we calculate the average of the sample proportions
con_avg = float(cal_conversion_rate(df,'purchase_TF'))
print(con_avg)

The conversion rate is:0.15065384615384617
0.15065384615384617


In [30]:
# In this case, we conduct a one-tailed test
def optimal_sample_size(confidence, power, df, df_A, df_B, target_variable):
    # get the mean
    x1 = cal_conversion_rate(df_A, target_variable)
    x2 = cal_conversion_rate(df_B, target_variable)
    x = cal_conversion_rate(df, target_variable)
    # get the variance 
    var1 = np.var(df_A[target_variable].tolist())
    var2 = np.var(df_B[target_variable].tolist())
    # calculate the size of sample
    size_1 = df_A.shape[0] 
    size_2 = df_B.shape[0]
    # calculate the value of minimum detectable effect
    mde = st.norm.ppf(power) * math.sqrt(var1/size_1 + var2/size_2) + (x1 - x2)
    # get the value of optimal sample size 
    n = (st.norm.ppf(confidence) * math.sqrt(2*x*(1-x)) + st.norm.ppf(power) * math.sqrt(x1*(1-x1)+x2*(1-x2)))**2/(mde **2)
    print('The optimal sample size is: %s' % n)
    return n

In [31]:
my_n = optimal_sample_size(confidence = 0.95, power = 0.8, df = df, df_A = df_A, df_B = df_B, target_variable = 'purchase_TF')

The conversion rate is:0.149616
The conversion rate is:0.1766
The conversion rate is:0.15065384615384617
The optimal sample size is: 3231.9989950223085


## Q2-2
### Conduct the test 10 times using the samples of the optimal size. Report results.

In [32]:
# now we get the optimal size is n, we round it to the ceil of it

op_n = math.ceil(my_n)
from random import sample
# creating a function to conduct the test 10 times using samples of the optimal size
def optimal_size_test(size, confidence,times, df_A, df_B, target_variable):
    for i in range(times):
        print('This is the No.%s test:' % (i+1))
        sample_df_A = df_A.sample(size)
        sample_df_B = df_B.sample(size)
        # get the mean
        x1 = cal_conversion_rate(sample_df_A, target_variable)
        x2 = cal_conversion_rate(sample_df_B, target_variable)
        # do A/B testing
        do_abtesting(pa = x1, pb = x2, b_size = size, confidence =  confidence)
        print('-'*40)

In [14]:
optimal_size_test(size = op_n , confidence = 0.95,times  = 10, df_A  = df_A, df_B = df_B, target_variable = 'purchase_TF')
   

This is the No.1 test:
The conversion rate is:0.15871559633027524
The conversion rate is:0.19541284403669726
z-score of variant b: 3.3156258803920493 > z-score of the confidence rate: 1.6448536269514722

We can reject null hypothesis given that confidence rate.

Therefore, implementing alternative B can improve the conversion rates over applying alternative A.
----------------------------------------
This is the No.2 test:
The conversion rate is:0.14128440366972478
The conversion rate is:0.20550458715596331
z-score of variant b: 6.087135989381587 > z-score of the confidence rate: 1.6448536269514722

We can reject null hypothesis given that confidence rate.

Therefore, implementing alternative B can improve the conversion rates over applying alternative A.
----------------------------------------
This is the No.3 test:
The conversion rate is:0.12844036697247707
The conversion rate is:0.18073394495412845
z-score of variant b: 5.1601494994954225 > z-score of the confidence rate: 1.6448536

## Q3-1
### Conduct a sequential test for the 10 sample, and find the average number of iterations required to stop the test.



In [18]:
import random
def conduct_sprt(times, size, alpha, power, df,df_A, df_B, target_variable):
    ln_A = math.log(1/alpha)
    ln_B = math.log(1-power)
    iteration = []
    for i in range(times):
        print('This is the No.%s test:' % (i+1))
        sample_df_A = df_A.sample(size)
        sample_df_B = df_B.sample(size)
        # get the mean
        pa = cal_conversion_rate(sample_df_A, target_variable)
        pb = cal_conversion_rate(sample_df_B, target_variable)
        # randomly shuffle data frame rows
        total_list = df['Variant'].tolist()
        random.shuffle(total_list)
        i = 0
        sum_log_lambda = 0
        while (sum_log_lambda > ln_B) and (sum_log_lambda < ln_A):
            if total_list[i] == 'B':
                sum_log_lambda += np.log(pa/pb)
            else:
                sum_log_lambda += np.log((1-pa)/(1-pb))
            i+=1
            if i >= len(total_list):
                print('We can not stop the test prior to using the full sample')
                break
        if sum_log_lambda <= ln_B:
            print('We fail to reject H0 in %s trials' % i)
        elif sum_log_lambda >= ln_A:
            print('We can reject H0 in %s trials' % i)
        print('-'*50)
        iteration.append(i)
    avg_iter = np.average(iteration)
    print('The average number of iterations required to stop the test is {}'.format(avg_iter))
    return avg_iter
                    
            
            

In [ ]:
conduct_sprt(times = 10, size = op_n , alpha = 0.05, power = 0.8, df = df,df_A = df_A, df_B = df_B, target_variable = 'purchase_TF')

This is the No.1 test:
The conversion rate is:0.14882425742574257
The conversion rate is:0.18471534653465346
We can reject H0 in 70 trials
--------------------------------------------------
This is the No.2 test:
The conversion rate is:0.14975247524752475
The conversion rate is:0.17790841584158415
We can reject H0 in 138 trials
--------------------------------------------------
This is the No.3 test:
The conversion rate is:0.14016089108910892
The conversion rate is:0.17264851485148514
